<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/%EC%9C%A0%EC%98%88%EC%8A%AC_LDA_LSA_%EB%A9%80%ED%8B%B0%EB%9D%BC%EB%B2%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 토픽 모델링 기반 멀티라벨 학습

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import re
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

# DATA_PATH = "/content/drive/MyDrive/생성 AI 모델링/data/"
DATA_PATH = "/content/drive/MyDrive/멀티캠퍼스 자료/Machine Learning/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device


'cuda'

In [3]:
# yogiyo =pd.read_csv(f"{DATA_PATH}yogiyo_reviews_jsi_all.csv") #18686개

# yogiyo =pd.read_csv(f"{DATA_PATH}yogiyo_reviews_0905_clean.csv") #24998개

yogiyo =pd.read_csv(f"{DATA_PATH}yogiyo_reviews_0908.csv") #37000개

In [4]:
yogiyo

,가게이름,고객리뷰,별점,맛별점,양별점,배달별점,사장댓글,메뉴
0,1988응답하라추억의옛날도시락-방이점,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,5,5.0,5.0,5.0,즐거운 순간을 저희와 함께해주셔서 감사합니다💖 \n\n보람차게 일할 수 있게 해 주...,[직화]삼겹옛날도시락
1,1988응답하라추억의옛날도시락-방이점,기름 있는 부분이 생각보다 많아요! 참고하세요:),5,5.0,5.0,5.0,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,"[직화]삼겹옛날도시락,볶음김치"
2,1988응답하라추억의옛날도시락-방이점,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,5,5.0,5.0,5.0,늘 최고의 메뉴를 제공해드리고 싶습니다!😄 \n\n맛있는 메뉴를 넉넉하게 먹을 수 ...,"[직화]삼겹옛날도시락,우삼겹청국장찌개＋대접밥"
3,1988응답하라추억의옛날도시락-방이점,"..,........",5,5.0,5.0,5.0,저희 매장과 함께 편안한 시간 되셨을까요? \n\n1. 고객을 생각하는 마음 \n2...,삼겹정찬도시락
4,1988응답하라추억의옛날도시락-방이점,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,5,5.0,4.0,5.0,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요!😁\n\n고객님의 행복...,"갈비옛날도시락＋미니냉면＋（서비스）콜라,스팸옛날도시락＋김찌 정식＋（서비스）콜라,부드러..."
...,...,...,...,...,...,...,...,...
37135,짬뽕관-전주금암점,맛있어요 ㅎ저번에주문하고 맛있어서또주문핰건데\n늦은시간에시켜서그렄지 국물이 엄청 걸...,5,5.0,5.0,5.0,고객님 안녕하세요 짬뽕관입니다~~^-^\n소중한 리뷰 너무 감사합니다!!🙂\n재주문...,짬뽕 ＋ 탕수육 set
37136,짬뽕관-전주금암점,맛있어요 또시킬게요,5,5.0,5.0,5.0,고객님 안녕하요 짬뽕관입니다!!^_^\n소중한 리뷰 남겨주셔 서 정말 달감사합니다!...,순두부짬뽕
37137,짬뽕관-전주금암점,정말 맛있게 잘먹었어요!!자주 시켜 먹을께요!,5,5.0,5.0,5.0,고객님 안녕하세요 짬뽕관입니다~~^_^\n맛있게 드셨다는 말씀에 힘이 나는거 같습니...,콩국수
37138,짬뽕관-전주금암점,와우.. 여기뭐죠 주변에서 맛있다고 먹어보라해서 시켰는데 육수가 일단 시원칼칼하니 ...,5,5.0,5.0,5.0,고객님 안녕하세요 짬뽕관입니다!!^_^\n맛있게 드셔주시고 정성스러운 사진 리뷰까지...,"차슈짬뽕,마파두부밥"


In [5]:
train= yogiyo[['고객리뷰','사장댓글','별점']]

# train = yogiyo[['review', 'reply','rating']]

rename_cols = ['review', 'reply','rating']
train.columns = rename_cols
train.head()

,review,reply,rating
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해주셔서 감사합니다💖 \n\n보람차게 일할 수 있게 해 주...,5
1,기름 있는 부분이 생각보다 많아요! 참고하세요:),소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,5
2,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,늘 최고의 메뉴를 제공해드리고 싶습니다!😄 \n\n맛있는 메뉴를 넉넉하게 먹을 수 ...,5
3,"..,........",저희 매장과 함께 편안한 시간 되셨을까요? \n\n1. 고객을 생각하는 마음 \n2...,5
4,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요!😁\n\n고객님의 행복...,5


In [6]:
review_list = []
for review in train['review']:
    if isinstance(review, str):

        pat = re.compile("[^a-zA-Z가-힣0-9.,~?!]")
        result = pat.sub(" ", review)

        result = re.sub(r'\.{2,}', '.', result)
        result = re.sub(r'\,{2,}', ',', result)
        result = re.sub(r'\~{2,}', '~', result)
        result = re.sub(r'\!{2,}', '!', result)
        result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        review_list.append(result)
    else:
        review_list.append("")

train['review'] = review_list
train.head()

<ipython-input-6-6374a3c305ac>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['review'] = review_list


,review,reply,rating
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해주셔서 감사합니다💖 \n\n보람차게 일할 수 있게 해 주...,5
1,기름 있는 부분이 생각보다 많아요! 참고하세요,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,5
2,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,늘 최고의 메뉴를 제공해드리고 싶습니다!😄 \n\n맛있는 메뉴를 넉넉하게 먹을 수 ...,5
3,".,.",저희 매장과 함께 편안한 시간 되셨을까요? \n\n1. 고객을 생각하는 마음 \n2...,5
4,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요!😁\n\n고객님의 행복...,5


In [7]:
reply_list = []
for reply in train['reply']:
    if isinstance(reply, str):

        pat = re.compile("[^a-zA-Z가-힣0-9.,~?!]")
        result = pat.sub(" ", reply)

        result = re.sub(r'\.{2,}', '.', result)
        result = re.sub(r'\,{2,}', ',', result)
        result = re.sub(r'\~{2,}', '~', result)
        result = re.sub(r'\!{2,}', '!', result)
        result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        reply_list.append(result)
    else:
        reply_list.append("")

train['reply'] = reply_list
train.head()

<ipython-input-7-2764c488e75c>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['reply'] = reply_list


,review,reply,rating
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,5
1,기름 있는 부분이 생각보다 많아요! 참고하세요,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,5
2,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,늘 최고의 메뉴를 제공해드리고 싶습니다! 맛있는 메뉴를 넉넉하게 먹을 수 있는 것보...,5
3,".,.",저희 매장과 함께 편안한 시간 되셨을까요? 1. 고객을 생각하는 마음 2. 더 나은...,5
4,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요! 고객님의 행복한 시간...,5


In [8]:
train['review']

0                                맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요
1                               기름 있는 부분이 생각보다 많아요! 참고하세요 
2          양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!
3                                                      .,.
4        냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...
                               ...                        
37135    맛있어요 저번에주문하고 맛있어서또주문핰건데 늦은시간에시켜서그렄지 국물이 엄청 걸쭉하...
37136                                           맛있어요 또시킬게요
37137                             정말 맛있게 잘먹었어요!자주 시켜 먹을께요!
37138    와우. 여기뭐죠 주변에서 맛있다고 먹어보라해서 시켰는데 육수가 일단 시원칼칼하니 조...
37139    제발 양파추가 선택란 하나 만들어주세요. 단무지대신 양파달라고 요청써놨지만 바쁠때 ...
Name: review, Length: 37140, dtype: object

## A. LDA (Latent Dirichlet Allocation)
-  주제 모델링에 사용/ 주어진 문서 집합에서 숨겨진 주제를 추출하고, 각 주제가 단어의 확률적 분포를 가지고 있다고 가정
- 참고 : https://heytech.tistory.com/401

### 1) 전체 리뷰에서 Topic 찾기

- 리뷰 리스트화 및 형태소 토큰화 (NNG,VA,VV,XR)

In [9]:
review_list = train['review'].tolist()
len(review_list)

37140

In [10]:
!pip install corpora

  Preparing metadata (setup.py) ... done
  Created wheel for corpora: filename=Corpora-1.0-py3-none-any.whl size=5495 sha256=aeec596e75a114f9004e576b32003106b0b8f6e1bb6825d38d772175757c7264
  Stored in directory: /root/.cache/pip/wheels/f4/84/89/9b605ada9b4fce8afd38b80a42d3a8836adaecdfc28bda687e
Successfully built corpora


In [11]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 37.0 MB/s eta 0:00:00


In [12]:
!pip install mecab-python3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.6/581.6 kB 5.8 MB/s eta 0:00:00


In [13]:
# echo "dicdir = /usr/local/lib/mecab/dic/mecab-ko-dic" | sudo tee -a /usr/local/etc/mecabrc
# 터미널에 작성

In [14]:
!pip install kiwipiepy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 17.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.16.0-py3-none-any.whl size=30813255 sha256=8d29265d4a9ed50073b93c2e68754e577e93eb3a1d145850e6056b337c7ae310
  Stored in directory: /root/.cache/pip/wheels/6b/03/2b/ac2c97cc65ebd9df3516f4b900adc2f0a744df8d1375b2e2ef
Successfully built kiwipiepy-model


In [15]:
from kiwipiepy import Kiwi

kiwi = Kiwi()

In [19]:
text = review_list[0]
result = kiwi.tokenize(text)
print(text)
result

맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요


[Token(form='맛있', tag='VA', start=0, len=2),
 Token(form='게', tag='EC', start=2, len=1),
 Token(form='잘', tag='MAG', start=4, len=1),
 Token(form='먹', tag='VV', start=5, len=1),
 Token(form='었', tag='EP', start=6, len=1),
 Token(form='습니다', tag='EC', start=7, len=3),
 Token(form='묵사발', tag='NNG', start=11, len=3),
 Token(form='도', tag='JX', start=14, len=1),
 Token(form='시원', tag='XR', start=16, len=2),
 Token(form='하', tag='XSA', start=18, len=1),
 Token(form='니', tag='EC', start=19, len=1),
 Token(form='맛있', tag='VA', start=21, len=2),
 Token(form='네요', tag='EF', start=23, len=2)]

In [20]:
# text = review_list[0]
# result = kiwi.analyze(text)
# print(text)
# result

In [21]:
[token.form for token in result if token.tag in ['NNG','VA','VV','XR']] #명사, 형용사, 동사, 어근


['맛있', '먹', '묵사발', '시원', '맛있']

- 전체 리뷰를 토큰화 진행

In [22]:
kiwi_reviews = []

for i, review in tqdm(enumerate(review_list)):
    result = kiwi.tokenize(review)
    tmp = [token.form for token in result if token.tag in ['NNG','VA','XR']]
    for i, word in enumerate(tmp):
        kiwi_reviews.append(word)

0it [00:00, ?it/s]

In [23]:
kiwi_reviews

['맛있',
 '묵사발',
 '시원',
 '맛있',
 '기름',
 '있',
 '부분',
 '생각',
 '많',
 '양',
 '푸짐',
 '맛있',
 '청국장',
 '냄새',
 '콩',
 '좋',
 '냉면',
 '미니',
 '냉면',
 '양',
 '적',
 '김치',
 '정식',
 '양',
 '적',
 '만족',
 '맛나',
 '맛',
 '양',
 '만족',
 '배달',
 '만족',
 '은근',
 '김치찌개',
 '된장찌개',
 '잡채',
 '처음',
 '맛있',
 '리뷰',
 '서비스',
 '괜찮',
 '다음',
 '맛있',
 '국물',
 '맛있',
 '맛있',
 '사진',
 '없',
 '맛나',
 '오늘',
 '묵사발',
 '짭',
 '고소',
 '맛있',
 '양',
 '푸짐',
 '맛있',
 '맛있',
 '돼지고기',
 '찌개',
 '돼지고기',
 '없',
 '마법',
 '리뷰',
 '테러',
 '선',
 '맛있',
 '맛있',
 '배달',
 '빠르',
 '맛있',
 '고기',
 '많',
 '배고프',
 '맛있',
 '맛있',
 '가격',
 '대비',
 '삼겹살',
 '양',
 '많',
 '푸짐하',
 '맛있',
 '이번',
 '아이',
 '잡채',
 '주문',
 '고기',
 '맛있',
 '서비스',
 '묵사발',
 '맛',
 '진짜',
 '예술',
 '담',
 '추가',
 '제육',
 '도시락',
 '맛있',
 '참고',
 '케찹',
 '맛',
 '양',
 '많',
 '맛있',
 '푸짐',
 '맛있',
 '맛있',
 '양',
 '많',
 '반찬',
 '맛있',
 '아이',
 '질',
 '반찬',
 '맛있',
 '맛있',
 '밥',
 '적',
 '그렇',
 '간',
 '느낌',
 '있',
 '김치찌개',
 '참치',
 '배달',
 '빠르',
 '늦',
 '시간',
 '가능',
 '좋',
 '제육',
 '지이인짜',
 '맛있',
 '맛나',
 '재주',
 '문의',
 '서비스',
 '묵사발',
 '감사',
 '맛있',
 '주문',
 '의사',
 '있',
 '

- LDA 토픽 모델링

In [24]:
import pandas as pd
import re
from gensim import corpora, models
from gensim.models import LdaModel
import pprint

In [25]:
documents = [kiwi_reviews]  # Wrap title_keyword inside another list
dictionary = corpora.Dictionary(documents)
corpus = [dictionary.doc2bow(document) for document in documents]


In [26]:
lda_model = models.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=10)


- 1차 : 가공 없이 토픽 모델링 -> 맛있다, 먹다, 시키다 좋다, 배달, 빠르다, 등 공통내용 제거 필요

In [27]:
topics = lda_model.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.095*"맛있" + 0.015*"좋" + 0.015*"맛" + 0.013*"배달" + 0.012*"양" + 0.010*"주문" + 0.009*"많" + 0.006*"빠르" + 0.006*"감사" + 0.005*"서비스"')
(1, '0.005*"맛있" + 0.002*"좋" + 0.001*"배달" + 0.001*"맛" + 0.001*"많" + 0.001*"양" + 0.001*"감사" + 0.001*"주문" + 0.001*"빠르" + 0.001*"같"')
(2, '0.031*"맛있" + 0.007*"좋" + 0.007*"배달" + 0.006*"양" + 0.006*"맛" + 0.005*"많" + 0.004*"빠르" + 0.004*"주문" + 0.004*"감사" + 0.003*"같"')
(3, '0.153*"맛있" + 0.035*"맛" + 0.031*"좋" + 0.029*"배달" + 0.026*"양" + 0.022*"많" + 0.019*"주문" + 0.016*"빠르" + 0.015*"감사" + 0.011*"같"')
(4, '0.008*"맛있" + 0.002*"양" + 0.002*"맛" + 0.002*"배달" + 0.002*"좋" + 0.001*"많" + 0.001*"감사" + 0.001*"주문" + 0.001*"같" + 0.001*"빠르"')


- 2차 : 공통어(불용어) 리스트 제거

In [28]:
stop_words = ['맛있','먹','맛','시키','좋','양','많','배달','빠르','주문','하','감사','같', #1차
              '다음','때','있','없','서비스','최고','리뷰','고기','피자','때', #2차
              '맛나','집','사진','굿','소스','오늘','만족','생각','밥','곳','사장','국물'] #3차

kiwi_reviews2 = []

for word in kiwi_reviews :
    if word not in stop_words :
        kiwi_reviews2.append(word)

In [29]:
len(kiwi_reviews), len(kiwi_reviews2)

(178464, 91758)

In [30]:
documents = [kiwi_reviews2]  # Wrap title_keyword inside another list
dictionary = corpora.Dictionary(documents)
corpus = [dictionary.doc2bow(document) for document in documents]

lda_model = models.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=10)

topics = lda_model.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.001*"음식" + 0.001*"푸짐" + 0.001*"김치" + 0.001*"처음" + 0.001*"추가" + 0.001*"추천" + 0.001*"요청" + 0.001*"시간" + 0.001*"포장" + 0.001*"깔끔"')
(1, '0.003*"처음" + 0.003*"깔끔" + 0.002*"번창" + 0.002*"시간" + 0.002*"푸짐" + 0.002*"치즈" + 0.002*"음식" + 0.002*"추가" + 0.002*"김치" + 0.002*"그렇"')
(2, '0.009*"음식" + 0.009*"처음" + 0.008*"깔끔" + 0.007*"시간" + 0.007*"추가" + 0.007*"푸짐" + 0.007*"포장" + 0.006*"요청" + 0.006*"번창" + 0.006*"추천"')
(3, '0.001*"처음" + 0.001*"음식" + 0.000*"깔끔" + 0.000*"요청" + 0.000*"포장" + 0.000*"늦" + 0.000*"사항" + 0.000*"번창" + 0.000*"초밥" + 0.000*"김치"')
(4, '0.002*"처음" + 0.002*"음식" + 0.002*"번창" + 0.002*"깔끔" + 0.002*"요청" + 0.002*"그렇" + 0.002*"푸짐" + 0.001*"김치" + 0.001*"초밥" + 0.001*"시간"')


- 하이퍼파라미터 포함

In [30]:
num_topics = 5
chunksize = 2000
passes = 20
iterations = 400
eval_every = None

temp = dictionary[0]
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every)

### 2) 긍정 vs 부정 리뷰에서 Topic 찾기

In [31]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.9 MB/s eta 0:00:00


In [32]:

from transformers import pipeline

# 모델 불러오기
model_name = "MLOpsEngineer/review-rate-prediction"
classifier = pipeline("sentiment-analysis", model=model_name)

In [33]:

import pandas as pd
from transformers import pipeline

def analyze_reviews(reviews):
    # 모델 불러오기
    model_name = "MLOpsEngineer/review-rate-prediction"
    classifier = pipeline("sentiment-analysis", model=model_name)

    results = []
    for review_text in tqdm(reviews):
        # 모델 추론
        result = classifier(review_text)[0]

        # 결과 해석
        label = result['label']
        score = result['score']

        if label in ['LABEL_0', 'LABEL_1']:
            sentiment = 0 # '부정'
        else:
            sentiment = 1 # '긍정'

        results.append((review_text, sentiment, score))

    # 데이터프레임 생성
    tmp = pd.DataFrame(results, columns=['review', 'sentiment', 'score'])
    return tmp

In [34]:
sentiment_train = analyze_reviews(train['review'])
sentiment_train

  0%|          | 0/37140 [00:00<?, ?it/s]

,review,sentiment,score
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,1,0.566494
1,기름 있는 부분이 생각보다 많아요! 참고하세요,0,0.723872
2,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,1,0.608408
3,".,.",0,0.499478
4,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,1,0.381793
...,...,...,...
37135,맛있어요 저번에주문하고 맛있어서또주문핰건데 늦은시간에시켜서그렄지 국물이 엄청 걸쭉하...,1,0.424215
37136,맛있어요 또시킬게요,1,0.720691
37137,정말 맛있게 잘먹었어요!자주 시켜 먹을께요!,1,0.833007
37138,와우. 여기뭐죠 주변에서 맛있다고 먹어보라해서 시켰는데 육수가 일단 시원칼칼하니 조...,1,0.900031


In [36]:
# sentiment_train.to_csv(f"{DATA_PATH}posneg_train_37000.csv",index=False)

In [ ]:
# sentiment_train = pd.read_csv(f"{DATA_PATH}posneg_train_37000.csv",index=False)

In [39]:
tmp = sentiment_train[['sentiment','score']]
train = pd.concat([train,tmp],axis=1)

In [40]:
train

,review,reply,rating,sentiment,score
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,5,1,0.566494
1,기름 있는 부분이 생각보다 많아요! 참고하세요,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,5,0,0.723872
2,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,늘 최고의 메뉴를 제공해드리고 싶습니다! 맛있는 메뉴를 넉넉하게 먹을 수 있는 것보...,5,1,0.608408
3,".,.",저희 매장과 함께 편안한 시간 되셨을까요? 1. 고객을 생각하는 마음 2. 더 나은...,5,0,0.499478
4,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요! 고객님의 행복한 시간...,5,1,0.381793
...,...,...,...,...,...
37135,맛있어요 저번에주문하고 맛있어서또주문핰건데 늦은시간에시켜서그렄지 국물이 엄청 걸쭉하...,고객님 안녕하세요 짬뽕관입니다~ 소중한 리뷰 너무 감사합니다! 재주문해 주셨는데 국...,5,1,0.424215
37136,맛있어요 또시킬게요,고객님 안녕하요 짬뽕관입니다! 소중한 리뷰 남겨주셔 서 정말 달감사합니다! 만족스러...,5,1,0.720691
37137,정말 맛있게 잘먹었어요!자주 시켜 먹을께요!,고객님 안녕하세요 짬뽕관입니다~ 맛있게 드셨다는 말씀에 힘이 나는거 같습니다 항상 ...,5,1,0.833007
37138,와우. 여기뭐죠 주변에서 맛있다고 먹어보라해서 시켰는데 육수가 일단 시원칼칼하니 조...,고객님 안녕하세요 짬뽕관입니다! 맛있게 드셔주시고 정성스러운 사진 리뷰까지 정말 감...,5,1,0.900031


#### 긍정 리뷰 토픽 모델링

In [45]:
mask = train['sentiment'] == 1
pos_train = train[mask] #35316개

pos_train

,review,reply,rating,sentiment,score
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,5,1,0.566494
2,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,늘 최고의 메뉴를 제공해드리고 싶습니다! 맛있는 메뉴를 넉넉하게 먹을 수 있는 것보...,5,1,0.608408
4,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요! 고객님의 행복한 시간...,5,1,0.381793
5,맛나게 잘 묵었습니다,소중한 순간을 저희 브랜드와 함께 해주셔서 영광입니다. 다음에도 고객님의 즐거운 순...,5,1,0.549870
6,맛도 양도 모두 만족합니다 배달도 정확히 보내 주시고 아주 만족합니다.,"안녕하세요! 주문해 주셔서 진심으로 감사드려요! 이렇게 좋은 리뷰 써 주시니, 메뉴...",5,1,0.778149
...,...,...,...,...,...
37135,맛있어요 저번에주문하고 맛있어서또주문핰건데 늦은시간에시켜서그렄지 국물이 엄청 걸쭉하...,고객님 안녕하세요 짬뽕관입니다~ 소중한 리뷰 너무 감사합니다! 재주문해 주셨는데 국...,5,1,0.424215
37136,맛있어요 또시킬게요,고객님 안녕하요 짬뽕관입니다! 소중한 리뷰 남겨주셔 서 정말 달감사합니다! 만족스러...,5,1,0.720691
37137,정말 맛있게 잘먹었어요!자주 시켜 먹을께요!,고객님 안녕하세요 짬뽕관입니다~ 맛있게 드셨다는 말씀에 힘이 나는거 같습니다 항상 ...,5,1,0.833007
37138,와우. 여기뭐죠 주변에서 맛있다고 먹어보라해서 시켰는데 육수가 일단 시원칼칼하니 조...,고객님 안녕하세요 짬뽕관입니다! 맛있게 드셔주시고 정성스러운 사진 리뷰까지 정말 감...,5,1,0.900031


In [48]:
pos_review_list = pos_train['review'].tolist()

In [50]:
pos_kiwi_reviews = []

for i, review in tqdm(enumerate(pos_review_list)):
    result = kiwi.tokenize(review)
    tmp = [token.form for token in result if token.tag in ['NNG','VA','XR']]
    for i, word in enumerate(tmp):
        pos_kiwi_reviews.append(word)

0it [00:00, ?it/s]

In [51]:
pos_kiwi_reviews

['맛있',
 '묵사발',
 '시원',
 '맛있',
 '양',
 '푸짐',
 '맛있',
 '청국장',
 '냄새',
 '콩',
 '좋',
 '냉면',
 '미니',
 '냉면',
 '양',
 '적',
 '김치',
 '정식',
 '양',
 '적',
 '만족',
 '맛나',
 '맛',
 '양',
 '만족',
 '배달',
 '만족',
 '은근',
 '김치찌개',
 '된장찌개',
 '잡채',
 '처음',
 '맛있',
 '리뷰',
 '서비스',
 '괜찮',
 '다음',
 '맛있',
 '국물',
 '맛있',
 '맛있',
 '사진',
 '없',
 '맛나',
 '오늘',
 '묵사발',
 '짭',
 '고소',
 '맛있',
 '양',
 '푸짐',
 '맛있',
 '맛있',
 '맛있',
 '맛있',
 '배달',
 '빠르',
 '맛있',
 '고기',
 '많',
 '배고프',
 '맛있',
 '맛있',
 '가격',
 '대비',
 '삼겹살',
 '양',
 '많',
 '푸짐하',
 '맛있',
 '이번',
 '아이',
 '잡채',
 '주문',
 '고기',
 '맛있',
 '서비스',
 '묵사발',
 '맛',
 '진짜',
 '예술',
 '담',
 '추가',
 '제육',
 '도시락',
 '맛있',
 '맛',
 '양',
 '많',
 '맛있',
 '푸짐',
 '맛있',
 '맛있',
 '양',
 '많',
 '반찬',
 '맛있',
 '아이',
 '질',
 '반찬',
 '맛있',
 '맛있',
 '밥',
 '적',
 '그렇',
 '간',
 '느낌',
 '있',
 '김치찌개',
 '참치',
 '배달',
 '빠르',
 '늦',
 '시간',
 '가능',
 '좋',
 '제육',
 '지이인짜',
 '맛있',
 '맛나',
 '재주',
 '문의',
 '서비스',
 '묵사발',
 '감사',
 '맛있',
 '주문',
 '의사',
 '있',
 '가격',
 '구성',
 '최고',
 '맛있',
 '맛있',
 '최고',
 '제육',
 '정식',
 '맛있',
 '김치찌개',
 '김치',
 '국',
 '느낌',
 '고기',
 '맛나',
 

- LDA 토픽 모델링

In [52]:
import pandas as pd
import re
from gensim import corpora, models
from gensim.models import LdaModel
import pprint

In [53]:
documents = [pos_kiwi_reviews]  # Wrap title_keyword inside another list
dictionary = corpora.Dictionary(documents)
corpus = [dictionary.doc2bow(document) for document in documents]

lda_model = models.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=10)

topics = lda_model.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.166*"맛있" + 0.034*"맛" + 0.032*"좋" + 0.029*"배달" + 0.027*"양" + 0.023*"많" + 0.019*"주문" + 0.017*"빠르" + 0.017*"감사" + 0.011*"같"')
(1, '0.004*"맛있" + 0.001*"맛" + 0.001*"좋" + 0.001*"배달" + 0.001*"양" + 0.001*"많" + 0.001*"주문" + 0.001*"감사" + 0.001*"빠르" + 0.000*"맛나"')
(2, '0.003*"맛있" + 0.001*"좋" + 0.001*"맛" + 0.001*"배달" + 0.001*"양" + 0.001*"많" + 0.001*"주문" + 0.000*"감사" + 0.000*"빠르" + 0.000*"같"')
(3, '0.005*"맛있" + 0.002*"맛" + 0.001*"양" + 0.001*"배달" + 0.001*"좋" + 0.001*"주문" + 0.001*"많" + 0.001*"빠르" + 0.001*"감사" + 0.001*"같"')
(4, '0.061*"맛있" + 0.024*"양" + 0.020*"배달" + 0.016*"맛" + 0.015*"많" + 0.014*"좋" + 0.012*"주문" + 0.009*"빠르" + 0.008*"감사" + 0.006*"같"')


In [54]:
stop_words = ['맛있','먹','맛','시키','좋','양','많','배달','빠르','주문','하','감사','같', #1차
              '다음','때','있','없','서비스','최고','리뷰','고기','피자','때', #2차
              '집','사진','굿','소스','오늘','만족','생각','밥','곳','사장','국물'] #3차

pos_kiwi_reviews2 = []

for word in pos_kiwi_reviews :
    if word not in stop_words :
        pos_kiwi_reviews2.append(word)

In [55]:
len(pos_kiwi_reviews), len(pos_kiwi_reviews2)

(161833, 80227)

In [56]:
documents = [pos_kiwi_reviews2]  # Wrap title_keyword inside another list
dictionary = corpora.Dictionary(documents)
corpus = [dictionary.doc2bow(document) for document in documents]

lda_model = models.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=10)

topics = lda_model.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.002*"처음" + 0.002*"음식" + 0.001*"깔끔" + 0.001*"초밥" + 0.001*"푸짐" + 0.001*"포장" + 0.001*"추가" + 0.001*"오랜만" + 0.001*"그렇" + 0.001*"김치"')
(1, '0.010*"처음" + 0.009*"음식" + 0.009*"깔끔" + 0.008*"푸짐" + 0.007*"추가" + 0.007*"포장" + 0.007*"번창" + 0.007*"시간" + 0.006*"추천" + 0.006*"요청"')
(2, '0.001*"푸짐" + 0.001*"처음" + 0.001*"번창" + 0.001*"음식" + 0.001*"메뉴" + 0.001*"추가" + 0.001*"앞" + 0.001*"시간" + 0.001*"김치" + 0.001*"오랜만"')
(3, '0.001*"처음" + 0.001*"푸짐" + 0.001*"깔끔" + 0.001*"음식" + 0.001*"포장" + 0.001*"추가" + 0.001*"추천" + 0.001*"시간" + 0.001*"가격" + 0.000*"오랜만"')
(4, '0.002*"처음" + 0.002*"깔끔" + 0.002*"요청" + 0.002*"음식" + 0.002*"추천" + 0.002*"번창" + 0.002*"시간" + 0.002*"초밥" + 0.001*"치즈" + 0.001*"메뉴"')


#### 부정 리뷰 토픽 모델링

In [46]:
mask = train['sentiment'] == 0
neg_train = train[mask] #1824개

neg_train

,review,reply,rating,sentiment,score
1,기름 있는 부분이 생각보다 많아요! 참고하세요,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,5,0,0.723872
3,".,.",저희 매장과 함께 편안한 시간 되셨을까요? 1. 고객을 생각하는 마음 2. 더 나은...,5,0,0.499478
17,"돼지고기찌개에 돼지고기가 없는 마법. 리뷰테러는 안할껀데 이건 선 넘은듯요.,.",안녕하세요 ha 님. 상세한 리뷰 주셔서 감사합니다. 주신 내용 잘 읽었습니다. 저...,5,0,0.822321
28,참고로 계란말이에 케찹 뿌려져 옴,오늘도 저희 매장에 귀한 발걸음을 해주셔서 영광입니다 오늘 뭐먹지? 라는 고민 끝에...,5,0,0.685938
48,"찌개 양은 많고 밥은 적당, 리뷰서비스는 조금만 덜 달면 더 맛있었겠어요.",소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...,5,0,0.455316
...,...,...,...,...,...
37094,짜지않게 부탁드렸는데요. 받아드려지지않았는지 좀 짰습니다. 탕수육 튀킴옷은 질기고 ...,저희 짬뽕관을 이용해 주셔서 감사합니다. 우선 식사에 만족시켜 드리지 못 해 정말 ...,4,0,0.990747
37095,시켜먹는 고객의 배려는 없습니다. 별도요청을 하였으나 국물이 그어느 중국집보다 현저...,저희 짬뽕관을 이용해 주셔서 감사합니다. 우선 즐거운 식사 시간에 불편을 드려 너무...,1,0,0.979012
37099,짜장 춘장맛만 나는 짬뽕 맛있게 매운 군만두 넹,저희 짬뽕관을 이용해 주셔서 감사합니다. 다음 주문은 더 맛있게 조리하여 별점 다섯...,4,0,0.581297
37111,탕수육은 비추. 돌 같은 식감 짬뽕은 맛있어요,저희 짬뽕관을 이용해 주셔서 감사합니다. 우선 식사시간에 아쉬움을 드려 정말 죄송합...,4,0,0.659974


In [57]:
neg_review_list = neg_train['review'].tolist()

In [58]:
neg_kiwi_reviews = []

for i, review in tqdm(enumerate(neg_review_list)):
    result = kiwi.tokenize(review)
    tmp = [token.form for token in result if token.tag in ['NNG','VA','XR']]
    for i, word in enumerate(tmp):
        neg_kiwi_reviews.append(word)

0it [00:00, ?it/s]

In [59]:
neg_kiwi_reviews

['기름',
 '있',
 '부분',
 '생각',
 '많',
 '돼지고기',
 '찌개',
 '돼지고기',
 '없',
 '마법',
 '리뷰',
 '테러',
 '선',
 '참고',
 '케찹',
 '찌개',
 '양',
 '많',
 '밥',
 '적당',
 '리뷰',
 '서비스',
 '조금',
 '맛있',
 '저번',
 '달',
 '맛있',
 '고기',
 '양',
 '된장찌개',
 '짜',
 '그렇',
 '맛',
 '있',
 '배달',
 '알림',
 '시간',
 '배달',
 '늦',
 '점',
 '고지',
 '없',
 '전화',
 '이유',
 '말씀',
 '배달',
 '기사',
 '픽업',
 '답',
 '제육',
 '갈비',
 '도시락',
 '김치찌개',
 '밍밍',
 '메뉴',
 '양',
 '맛',
 '괜찮',
 '전체',
 '아쉬움',
 '크',
 '뒤늦',
 '리뷰',
 '사과',
 '답',
 '리뷰',
 '배상',
 '앞',
 '일',
 '없',
 '같',
 '괜찮',
 '맛',
 '청국장',
 '짜',
 '공기',
 '밥',
 '추가',
 '맛',
 '양',
 '무난하',
 '배달',
 '예정',
 '시간',
 '무색하',
 '늦',
 '도착',
 '배달',
 '예정',
 '시간',
 '여유',
 '배달',
 '예정',
 '시간',
 '메인',
 '좋',
 '반찬',
 '촉감',
 '맛',
 '이상',
 '묵',
 '오뎅',
 '김치',
 '나물',
 '무',
 '예정',
 '시간',
 '도착',
 '음식',
 '배달',
 '느리',
 '감점',
 '배',
 '고프',
 '그렇',
 '맛',
 '개취',
 '존중',
 '맛있',
 '서비스',
 '맛있',
 '배달',
 '늦',
 '옴',
 '전화',
 '전화',
 '도시락',
 '집',
 '맛나',
 '집',
 '아이',
 '리뷰',
 '이벤트',
 '같',
 '적',
 '배달',
 '전화',
 '배달',
 '여자',
 '한숨',
 '비',
 '배달',
 '기사',
 '이해',
 '사람',
 '크',
 '돈',

- LDA 토픽 모델링

In [60]:
import pandas as pd
import re
from gensim import corpora, models
from gensim.models import LdaModel
import pprint

In [61]:
documents = [neg_kiwi_reviews]  # Wrap title_keyword inside another list
dictionary = corpora.Dictionary(documents)
corpus = [dictionary.doc2bow(document) for document in documents]

lda_model = models.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=10)

topics = lda_model.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.002*"맛" + 0.001*"배달" + 0.001*"맛있" + 0.001*"주문" + 0.001*"좋" + 0.001*"없" + 0.001*"같" + 0.001*"양" + 0.001*"리뷰" + 0.001*"고기"')
(1, '0.002*"맛" + 0.002*"배달" + 0.001*"양" + 0.001*"맛있" + 0.001*"좋" + 0.001*"같" + 0.001*"없" + 0.001*"주문" + 0.001*"리뷰" + 0.001*"있"')
(2, '0.044*"맛" + 0.031*"배달" + 0.027*"맛있" + 0.020*"같" + 0.018*"없" + 0.018*"좋" + 0.017*"양" + 0.016*"주문" + 0.011*"많" + 0.010*"고기"')
(3, '0.004*"맛" + 0.003*"배달" + 0.002*"맛있" + 0.002*"같" + 0.002*"좋" + 0.002*"양" + 0.002*"없" + 0.001*"주문" + 0.001*"고기" + 0.001*"적"')
(4, '0.002*"맛" + 0.002*"배달" + 0.001*"맛있" + 0.001*"양" + 0.001*"같" + 0.001*"좋" + 0.001*"주문" + 0.001*"없" + 0.001*"고기" + 0.001*"있"')


In [62]:
stop_words = ['맛있','먹','맛','시키','좋','양','많','배달','빠르','주문','하','감사','같', #1차
              '다음','때','있','없','서비스','최고','리뷰','고기','피자','때', #2차
              '집','사진','굿','소스','오늘','만족','생각','밥','곳','사장','국물'] #3차

neg_kiwi_reviews2 = []

for word in neg_kiwi_reviews :
    if word not in stop_words :
        neg_kiwi_reviews2.append(word)

In [63]:
len(neg_kiwi_reviews), len(neg_kiwi_reviews2)

(16631, 11543)

In [65]:
documents = [neg_kiwi_reviews2]  # Wrap title_keyword inside another list
dictionary = corpora.Dictionary(documents)
corpus = [dictionary.doc2bow(document) for document in documents]

lda_model = models.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=10)

topics = lda_model.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.002*"적" + 0.002*"시간" + 0.001*"음식" + 0.001*"그렇" + 0.001*"전화" + 0.001*"느낌" + 0.001*"전" + 0.001*"이번" + 0.001*"늦" + 0.001*"괜찮"')
(1, '0.012*"적" + 0.011*"음식" + 0.011*"시간" + 0.011*"그렇" + 0.008*"추가" + 0.008*"전화" + 0.007*"느낌" + 0.007*"늦" + 0.007*"괜찮" + 0.006*"정도"')
(2, '0.001*"적" + 0.001*"전화" + 0.001*"시간" + 0.001*"음식" + 0.001*"추가" + 0.001*"그렇" + 0.001*"느낌" + 0.001*"괜찮" + 0.001*"늦" + 0.001*"포장"')
(3, '0.001*"음식" + 0.001*"적" + 0.001*"시간" + 0.001*"그렇" + 0.001*"면" + 0.001*"늦" + 0.001*"정도" + 0.001*"괜찮" + 0.001*"추가" + 0.001*"신경"')
(4, '0.001*"시간" + 0.001*"음식" + 0.001*"그렇" + 0.001*"적" + 0.001*"추가" + 0.001*"전화" + 0.001*"느낌" + 0.001*"괜찮" + 0.001*"실망" + 0.001*"늦"')


## B. LSA (Latent Semantic Analysis)
- 텍스트 문서 간의 의미적 유사성을 분석하고 단어의 의미를 추출하기 위해 사용 / 단어-문서 행렬 또는 단어-단어 행렬을 통해 의미를 추출하고 다차원 공간에서 차원 축소를 수행

## C. 토픽매칭 사전학습 모델


In [47]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# 모델과 토크나이저 로드
model_name = "amberoad/bert-multilingual-passage-reranking-msmarco"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)



주제 '맛'에 대한 관련성 점수:
- 맛있어요. 정말 맛있어요.: 0.04013018310070038
- 양이 푸짐하고 좋아요.: -2.338395357131958
- 가격 대비 맛있어요.: -2.6419355869293213
- 서비스가 빠르고 좋아요.: -2.5020735263824463
- 맛과 가격이 모두 좋아요.: -2.3940234184265137
주제 '양'에 대한 관련성 점수:
- 맛있어요. 정말 맛있어요.: -2.4318349361419678
- 양이 푸짐하고 좋아요.: -1.7715847492218018
- 가격 대비 맛있어요.: -2.6851212978363037
- 서비스가 빠르고 좋아요.: -2.7551934719085693
- 맛과 가격이 모두 좋아요.: -2.464407205581665
주제 '가격'에 대한 관련성 점수:
- 맛있어요. 정말 맛있어요.: -2.8548901081085205
- 양이 푸짐하고 좋아요.: -2.373436689376831
- 가격 대비 맛있어요.: -2.3411905765533447
- 서비스가 빠르고 좋아요.: -2.6248505115509033
- 맛과 가격이 모두 좋아요.: -2.060382843017578
주제 '서비스'에 대한 관련성 점수:
- 맛있어요. 정말 맛있어요.: -2.8132286071777344
- 양이 푸짐하고 좋아요.: -3.001352548599243
- 가격 대비 맛있어요.: -2.8289682865142822
- 서비스가 빠르고 좋아요.: -2.074077844619751
- 맛과 가격이 모두 좋아요.: -3.0835487842559814


In [68]:
# 리뷰 데이터
reviews = [
    "맛있어요. 정말 맛있어요.",
    "양이 너무 적어요ㅠㅠ.",
    "가격 대비 맛있어요.",
    "사장님이 친절해요.",
    "배달이 너무 느려요ㅠㅠ."
]

# 주제별 쿼리 설정
queries = ["맛", "양", "가격", "서비스","배달"]

# 주제별 관련성 점수를 저장할 리스트
scores_by_topic = []

# 각 주제별로 모델 예측 수행
for query in queries:
    # 리뷰와 주제별 쿼리를 토큰화
    inputs = tokenizer(reviews, [query] * len(reviews), return_tensors="pt", padding=True, truncation=True, max_length=128)

    # 모델 예측
    with torch.no_grad():
        logits = model(**inputs).logits

    # 관련성 점수 추출
    scores = logits[:, 1].tolist()  # 레이블 1에 대한 점수를 추출 (관련성 점수)
    scores_by_topic.append(scores)

# 주제별 관련성 점수 출력
for i, query in enumerate(queries):
    print(f"주제 '{query}'에 대한 관련성 점수:")
    for j, review in enumerate(reviews):
        print(f"- {query} : {review}: {scores_by_topic[i][j]}")



주제 '맛'에 대한 관련성 점수:
- 맛 : 맛있어요. 정말 맛있어요.: 0.04013018310070038
- 맛 : 양이 너무 적어요ㅠㅠ.: -2.5560219287872314
- 맛 : 가격 대비 맛있어요.: -2.6419355869293213
- 맛 : 사장님이 친절해요.: -2.5285427570343018
- 맛 : 배달이 너무 느려요ㅠㅠ.: -2.5928356647491455
주제 '양'에 대한 관련성 점수:
- 양 : 맛있어요. 정말 맛있어요.: -2.4318349361419678
- 양 : 양이 너무 적어요ㅠㅠ.: -1.9880471229553223
- 양 : 가격 대비 맛있어요.: -2.6851212978363037
- 양 : 사장님이 친절해요.: -2.3066041469573975
- 양 : 배달이 너무 느려요ㅠㅠ.: -2.2473666667938232
주제 '가격'에 대한 관련성 점수:
- 가격 : 맛있어요. 정말 맛있어요.: -2.8548901081085205
- 가격 : 양이 너무 적어요ㅠㅠ.: -2.2161152362823486
- 가격 : 가격 대비 맛있어요.: -2.3411905765533447
- 가격 : 사장님이 친절해요.: -3.6042325496673584
- 가격 : 배달이 너무 느려요ㅠㅠ.: -2.411973237991333
주제 '서비스'에 대한 관련성 점수:
- 서비스 : 맛있어요. 정말 맛있어요.: -2.8132286071777344
- 서비스 : 양이 너무 적어요ㅠㅠ.: -2.9625461101531982
- 서비스 : 가격 대비 맛있어요.: -2.8289682865142822
- 서비스 : 사장님이 친절해요.: -3.336125612258911
- 서비스 : 배달이 너무 느려요ㅠㅠ.: -2.7229180335998535
주제 '배달'에 대한 관련성 점수:
- 배달 : 맛있어요. 정말 맛있어요.: -2.4630634784698486
- 배달 : 양이 너무 적어요ㅠㅠ.: -2.4196958541870117
- 배달

In [86]:
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# 리뷰 데이터
reviews = [
    "맛있어요. 정말 맛있어요.",
    "양이 너무 적어요ㅠㅠ.",
    "가격 대비 맛있어요.",
    "사장님이 친절해요.",
    "배달이 너무 느려요ㅠㅠ."
]

# 주제별 쿼리 설정
queries = ["맛", "양", "가격", "서비스", "배달"]

# 주제별 관련성 점수를 저장할 리스트
scores_by_topic = []

# 최고 관련성 주제를 저장할 리스트
best_topics = []

# 모델과 토크나이저 로드
model_name = "amberoad/bert-multilingual-passage-reranking-msmarco"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# 각 주제별로 모델 예측 수행
for query in queries:
    # 리뷰와 주제별 쿼리를 토큰화
    inputs = tokenizer(reviews, [query] * len(reviews), return_tensors="pt", padding=True, truncation=True, max_length=2048)

    # 모델 예측
    with torch.no_grad():
        logits = model(**inputs).logits

    # 관련성 점수 추출
    scores = logits[:, 1].tolist()  # 레이블 1에 대한 점수를 추출 (관련성 점수)
    scores_by_topic.append(scores)

    # 가장 높은 관련성 주제 찾기
    best_topic = queries[scores.index(max(scores))]
    best_topics.append(best_topic)

# 결과를 데이터프레임으로 저장
data = {'리뷰': reviews}
for i, query in enumerate(queries):
    data[f"{query} 관련도"] = scores_by_topic[i]
data['최고 관련 주제'] = best_topics

df = pd.DataFrame(data)

# 데이터프레임 출력
df


,리뷰,맛 관련도,양 관련도,가격 관련도,서비스 관련도,배달 관련도,최고 관련 주제
0,맛있어요. 정말 맛있어요.,0.040130,-2.431835,-2.854890,-2.813229,-2.463063,맛
1,양이 너무 적어요ㅠㅠ.,-2.556022,-1.988047,-2.216115,-2.962546,-2.419696,양
2,가격 대비 맛있어요.,-2.641936,-2.685121,-2.341191,-2.828968,-2.753579,양
3,사장님이 친절해요.,-2.528543,-2.306604,-3.604233,-3.336126,-2.458638,배달
4,배달이 너무 느려요ㅠㅠ.,-2.592836,-2.247367,-2.411973,-2.722918,-1.475893,배달


In [79]:
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

def calculate_review_scores(reviews):
    # 주제별 쿼리 설정
    queries = ["맛", "양", "가격", "서비스", "배달"]

    # 주제별 관련성 점수를 저장할 리스트
    scores_by_topic = []

    # 최고 관련성 주제를 저장할 리스트
    best_topics = []

    # 모델과 토크나이저 로드
    model_name = "amberoad/bert-multilingual-passage-reranking-msmarco"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)

    # 각 주제별로 모델 예측 수행
    for query in tqdm(queries):
        # 리뷰와 주제별 쿼리를 토큰화
        inputs = tokenizer(reviews, [query] * len(reviews), return_tensors="pt", padding=True, truncation=True, max_length=128)

        # 모델 예측
        with torch.no_grad():
            logits = model(**inputs).logits

        # 관련성 점수 추출
        scores = logits[:, 1].tolist()  # 레이블 1에 대한 점수를 추출 (관련성 점수)
        scores_by_topic.append(scores)

        # 가장 높은 관련성 주제 찾기
        best_topic = queries[scores.index(max(scores))]
        best_topics.append(best_topic)

    # 결과를 데이터프레임으로 저장
    data = {'리뷰': reviews}
    for i, query in enumerate(queries):
        data[f"{query} 관련도"] = scores_by_topic[i]
    data['최고 관련 주제'] = best_topics

    df = pd.DataFrame(data)
    return df


In [81]:
neg_review_list[:20]

['기름 있는 부분이 생각보다 많아요! 참고하세요 ',
 '.,.',
 '돼지고기찌개에 돼지고기가 없는 마법. 리뷰테러는 안할껀데 이건 선 넘은듯요.,.',
 '참고로 계란말이에 케찹 뿌려져 옴',
 '찌개 양은 많고 밥은 적당, 리뷰서비스는 조금만 덜 달면 더 맛있었겠어요.',
 '저저번달에 맛있게 먹어서 또 시켰는데 고기 양이 엄청 줄었네요 .된장찌개도 엄청 짜고 그래도 맛은 있었어요',
 '20분 이내 배달 알림 주시고는 한시간이 지나서야 받았습니다. 배달이 늦는 점에 대한 고지가 없어 전화드리니 이유는 말씀 없이 배달기사분이 픽업했다는 답만 있었구요. 제육 시켰는데 갈비도시락이 오고, 김치찌개는 밍밍하고 식어있어 많이 아쉬웠습니다 다른 메뉴들의 양이나 맛은 괜찮았지만 전체적으로 아쉬움이 크고, 뒤늦게 다른 리뷰들을 보니 사과 답리뷰만 있을 뿐 배상이 되는 것도 아닌지라 앞으로 시킬 일은 없을 것같습니다.',
 '괜찮았어요.',
 '맛은잇는데 청국장이넘짜요 글구 공기밥추가햇는데 안왓어요',
 '맛이나 양은 무난해요 그런데. 배달예정시간이 무색하게 늦게도착 차라리 배달 예정시간을 여유있게 잡던지. 배달예정시간이 지나면 계속 기다리게되니 ',
 '메인은 좋았는데, 아. 반찬이 다 쉬어 모두 버렸어요 한번 씹는데 촉감,맛이 이상해요 묵,오뎅,김치,나물,무.',
 '예정시간보다 30분이 지나서야 도착하고 음식도 다 식어서 왔네요',
 '배달이 넘 느려서 우선 감점 배가 고픈데 그럼에도 맛은 별로 개취존중~스팸이 젤 맛있고 서비스가 더 맛있어요',
 '배달 늦게옴 전화주면 받으러나간다니깐 전화 안줌',
 '도시락집붕 맛난집아이이이이',
 '리뷰 이벤트 한다지만 너무 아닌 거 같아서 적습니다. 배달이 20분 넘게 안오길래 전화해서 배달이 안온다 했더니 여자분께서 한숨 푹 쉬시면서 하아~비가 와서 배달 기사 안 잡힌다 하시는데 제가 뭐 그런거에 이해 못하는 사람도 아니고 그냥 안와서 물어보는건데 한숨음 왜 크게 쉬시나요? 아니 돈을 내고 배달 시간도 20분이나 지나서

In [83]:
# 함수 호출 예시
reviews = neg_review_list[5]

result_df = calculate_review_scores(reviews)
result_df

  0%|          | 0/5 [00:00<?, ?it/s]

ValueError: ignored

In [74]:
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

def calculate_review_scores(reviews):
    # 주제별 쿼리 설정
    queries = ["맛", "양", "가격", "서비스", "배달"]

    # 주제별 관련성 점수를 저장할 리스트
    scores_by_topic = {query: [] for query in queries}

    # 최고 관련성 주제를 저장할 리스트
    best_topics = []

    # 모델과 토크나이저 로드
    model_name = "amberoad/bert-multilingual-passage-reranking-msmarco"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)

    # 각 주제별로 모델 예측 수행
    for query in queries:
        # 리뷰와 주제별 쿼리를 토큰화
        inputs = tokenizer(reviews, [query] * len(reviews), return_tensors="pt", padding=True, truncation=True, max_length=128)

        # 모델 예측
        with torch.no_grad():
            logits = model(**inputs).logits

        # 관련성 점수 추출
        scores = logits[:, 1].tolist()  # 레이블 1에 대한 점수를 추출 (관련성 점수)
        scores_by_topic[query] = scores

    # 가장 높은 관련성 주제 찾기
    for i in range(len(reviews)):
        best_topic = queries[0]  # 초기값 설정
        min_score_difference = float('inf')  # 초기값 설정

        # 모든 주제 간의 관련도 차이 비교
        for query1 in queries:
            for query2 in queries:
                if query1 != query2:
                    score_difference = abs(scores_by_topic[query1][i] - scores_by_topic[query2][i])
                    if score_difference < min_score_difference:
                        min_score_difference = score_difference
                        best_topic = query1 if scores_by_topic[query1][i] > scores_by_topic[query2][i] else query2

        best_topics.append(best_topic)

    # 결과를 데이터프레임으로 저장
    data = {'리뷰': reviews}
    for query in queries:
        data[f"{query} 관련도"] = scores_by_topic[query]
    data['최고 관련 주제'] = best_topics

    df = pd.DataFrame(data)
    return df

# 함수 호출 예시
reviews = [
    "맛있어요. 정말 맛있어요.",
    "양이 너무 적어요ㅠㅠ.",
    "가격 대비 맛있어요.",
    "사장님이 친절해요.",
    "배달이 너무 느려요ㅠㅠ."
]

result_df = calculate_review_scores(reviews)
print(result_df)


               리뷰     맛 관련도     양 관련도    가격 관련도   서비스 관련도    배달 관련도 최고 관련 주제
0  맛있어요. 정말 맛있어요.  0.040130 -2.431835 -2.854890 -2.813229 -2.463063        양
1    양이 너무 적어요ㅠㅠ. -2.556022 -1.988047 -2.216115 -2.962546 -2.419696       배달
2     가격 대비 맛있어요. -2.641936 -2.685121 -2.341191 -2.828968 -2.753579        맛
3      사장님이 친절해요. -2.528543 -2.306604 -3.604233 -3.336126 -2.458638       배달
4   배달이 너무 느려요ㅠㅠ. -2.592836 -2.247367 -2.411973 -2.722918 -1.475893        맛
